In [1]:
import pandas as pd
import glob
import re

In [2]:
meta=pd.read_csv("meta.txt", sep="\t")

In [ ]:
out=meta[["htid", "pub_place", "title", "author", "rights_date_used"]]

In [59]:
head="ElocutionBooks/*"

In [61]:
text=[]
for f in glob.glob(head):
    with open(f, 'r', encoding="utf-8") as file:
        data = file.read().replace('\n', '')
        text.append(data)

In [118]:
out["text"]=text

<ipython-input-118-7bf8f31f12d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out["text"]=text


In [130]:
#keeps only alpha characters, removes hathi formatting, and sets to lowercase
def clean(text):
    try:
        s=text.split("Generated at University of Illinois at Urbana-Champaign on")[1]
    except:
        s=text
    s=re.sub(r'#*#', '', s) #remove page delims
    s=re.sub('[^a-zA-Z ]+', '', s) #keep only alphabetical characters
    s=' '.join(s.split())
    
    s=s.lower()
    s=s.replace("gmt", "")
    s=s.replace(" p ", "")
    s=s[200:] #remove first 200 characters, boilerplace, publication, added info etc
    return s

In [131]:
out["text"]=out["text"].apply(clean)

<ipython-input-131-9e5710170f08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out["text"]=out["text"].apply(clean)


In [137]:
from internetarchive import get_item, download, search_items

C:\Users\lucia\anaconda3\envs\mecp2\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.1) or chardet (3.0.4)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [145]:
search="title:"+"elocution"

In [159]:
ia_books=[]
count=0

#takes first 100 results of search between 1800 and 1900
for i in search_items(search):
    if count>100:
        break
    item = get_item(i["identifier"])
    meta=item.item_metadata['metadata']
    id=i["identifier"]
    try:
        date=int(meta["date"])
    except:
        date=0
    title=meta["title"]
    try:
        author=meta["creator"]
    except:
        author=""
    if date<1900 and date>1800:
        count+=1
        ia_books.append([id, "", title, author, date])
        #download(id, verbose=False, glob_pattern='*txt', destdir="ia_books/")
    

1855
1890
0
1890
0
1895
0
1820
1942
1873
1842
1806
1853
1850
1803
1903
0
1979
1922
1896
1898
1832
1833
1830
1830
1830
1890
1890
1903
1903
1871
1878
1865
1867
1884
1838
1884
0
1787
1787
1886
1845
1834
0
1856
1834
1851
1839
1850
1849
1836
1834
1880
1883
1959
1852
1846
0
1862
1867
1847
1847
0
1861
0
0
1841
1878
1841
1843
1853
0
1892
0
1883
1791
1833
1911
1820
0
1867
1897
1818
1872
1869
1878
1876
1860
1800
1900
1886
0
1912
1884
1880
1896
1882
1885
1857
1871
1803
1968
1896
1893
1887
1890
1890
1903
1893
1883
1868
1870
1871
1874
1896
1899
1896
1890
1878
1878
1886
1888
1872
1886
1881
1871
1896
1881
1886
1865
1897


In [176]:
ia=pd.DataFrame(ia_books)

In [178]:
ia.columns=["htid", "pub_place", "title", "author", "rights_date_used"]

In [172]:
text=[]
for i in ia[0]:
    try:
        f=glob.glob("ia_books/"+i+"/*.txt")[0]
    except:
        text.append("")
        continue
    with open(f, 'r', encoding="utf-8") as file:
        data = file.read().replace('\n', '')
        
        text.append(clean(data))

In [179]:
ia["text"]=text

In [181]:
full=out.append(ia)

In [182]:
full=full[full['text'].str.len()>500]

In [ ]:
g.drop_duplicates(subset=['brand', 'style'], keep='last')

In [184]:
full.to_csv("hathi_ia_texts.csv")

In [186]:
out.to_csv("hathi_only.csv")

In [190]:
full["title"].value_counts()

A system of elocution, with special reference to gesture, to the treatment of stammering, and defective articulation ..                                                                                                                                          3
Advanced elocution;                                                                                                                                                                                                                                              2
An Essay on Elocution: With Elucidatory Passages from Various Authors to which are Added Remarks ...                                                                                                                                                             2
Public speaking and reading; a treatise on delivery according to the principles of the new elocution                                                                                                                           